In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import time
import math
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
seed = 2020

In [ ]:
data_answer = pd.read_csv('data_answer_4.csv')
before=[line for line in data_answer['before'] if pd.isnull(line)==False]     
after=[line for line in data_answer['after'] if pd.isnull(line)==False]

trans=data_answer[data_answer['trans_count']!=0]     # Extract 75000 pieces of data from trans-data randomly as part of training set. The rest is used as a test set.
notrans=data_answer[data_answer['trans_count']==0].sample(75000,replace=False,axis=0)     # Extract 75000 pieces of data from untrans-data randomly.
trans

In [5]:
train_pd=pd.concat([trans[:75000],notrans],axis=0)     # Combine 75000 trans-data and 75000 untrans-data into a training set.
train_pd=train_pd.sample(frac=1)     # shuffle
train_before=[line for line in train_pd['before'] if pd.isnull(line)==False]
train_after=[line for line in train_pd['after'] if pd.isnull(line)==False]
train_before[0]

'另一方面 RDBMS 很 难 管理 会 话 中 对象 间 动态 关系 的 复杂性 而 OODBMS 是 基于 动态 模式 的'

In [25]:
# Construct a training set split by char.
before_train_list = []
after_train_list = []
for s in train_before:
    sl=[]
    for char in s:
        for c in char:
            if c !=' ':
#                 print(c)
                sl.append(c)
    before_train_list.append(sl)
for s in train_after:
    sl=[]
    for char in s:
        for c in char:
            if c !=' ':
#                 print(c)
                sl.append(c)
    after_train_list.append(sl)
before_train_list[0]

['另',
 '一',
 '方',
 '面',
 'R',
 'D',
 'B',
 'M',
 'S',
 '很',
 '难',
 '管',
 '理',
 '会',
 '话',
 '中',
 '对',
 '象',
 '间',
 '动',
 '态',
 '关',
 '系',
 '的',
 '复',
 '杂',
 '性',
 '而',
 'O',
 'O',
 'D',
 'B',
 'M',
 'S',
 '是',
 '基',
 '于',
 '动',
 '态',
 '模',
 '式',
 '的']

In [28]:
# char-num
before_num_data = [[before2id[b] for b in line ] for line in before_train_list]
after_num_data = [[after2id[a] for a in line] for line in after_train_list]

In [30]:
np.save('before_num_data_0.npy',before_num_data)
np.save('after_num_data_0.npy',after_num_data)

D:\Codetools\python3.9\lib\site-packages\numpy\lib\npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [ ]:
data_answer = pd.read_csv('data_answer_4.csv')
before=[line for line in data_answer['before'] if pd.isnull(line)==False]     
after=[line for line in data_answer['after'] if pd.isnull(line)==False]

# <eos>
before_token_list = [[char for char in line]+["<eos>"] for line in before]
after_token_list = [[char for char in line]+["<eos>"] for line in after]

# basic dict
basic_dict = {'<pad>':0, '<unk>':1, '<bos>':2, '<eos>':3}
# before-id,id-before
before_vocab = set(''.join(before))
before2id = {char:i+len(basic_dict) for i, char in enumerate(before_vocab)}
before2id.update(basic_dict)
id2before = {v:k for k,v in before2id.items()}

# after-id,id-after
after_vocab = set(''.join(after))
after2id = {char:i+len(basic_dict) for i, char in enumerate(after_vocab)}
after2id.update(basic_dict)
id2after = {v:k for k,v in after2id.items()}

# char-num
before_num_data = [[before2id[b] for b in line ] for line in before_token_list]
after_num_data = [[after2id[a] for a in line] for line in after_token_list]

print('char:', before[0])
print('index:', before_num_data[0])

In [6]:
#================dict================#
before2id=np.load('before2id_dict_0.npy',allow_pickle=True).item()
after2id=np.load('after2id_dict_0.npy',allow_pickle=True).item()
id2before=np.load('id2before_dict_0.npy',allow_pickle=True).item()
id2after=np.load('id2after_dict_0.npy',allow_pickle=True).item()
basic_dict = {'<pad>':0, '<unk>':1, '<bos>':2, '<eos>':3}

In [7]:
#================training_set================#
before_num_data=np.load('before_num_data_eos0.npy',allow_pickle=True)
before_num_data=list(before_num_data)
after_num_data=np.load('after_num_data_eos0.npy',allow_pickle=True)
after_num_data=list(after_num_data)
print('index:', before_num_data[0])

index: [6586, 4629, 1138, 2171, 2113, 374, 4139, 2429, 6114, 1739, 5739, 5726, 2554, 1325, 5126, 5025, 1008, 5013, 1743, 2012, 2563, 284, 4710, 1427, 5828, 2029, 4521, 4641, 6040, 6040, 374, 4139, 2429, 6114, 5760, 5221, 2405, 2012, 2563, 1573, 4847, 1427, 3]


In [8]:
class TranslationDataset(Dataset):
    def __init__(self, src_data, trg_data):
        self.src_data = src_data
        self.trg_data = trg_data

        assert len(src_data) == len(trg_data), \
            "numbers of src_data  and trg_data must be equal!"

    def __len__(self):
        return len(self.src_data)

    def __getitem__(self, idx):
        src_sample =self.src_data[idx]
        src_len = len(self.src_data[idx])
        trg_sample = self.trg_data[idx]
        trg_len = len(self.trg_data[idx])
        return {"src": src_sample, "src_len": src_len, "trg": trg_sample, "trg_len": trg_len}
    
def padding_batch(batch):
    """
    input: -> list of dict
        [{'src': [1, 2, 3], 'trg': [1, 2, 3]}, {'src': [1, 2, 2, 3], 'trg': [1, 2, 2, 3]}]
    output: -> dict of tensor 
        {
            "src": [[1, 2, 3, 0], [1, 2, 2, 3]].T
            "trg": [[1, 2, 3, 0], [1, 2, 2, 3]].T
        }
    """
    src_lens = [d["src_len"] for d in batch]
    trg_lens = [d["trg_len"] for d in batch]
    
    src_max = max([d["src_len"] for d in batch])
    trg_max = max([d["trg_len"] for d in batch])
    for d in batch:
        d["src"].extend([before2id["<pad>"]]*(src_max-d["src_len"]))
        d["trg"].extend([after2id["<pad>"]]*(trg_max-d["trg_len"]))
    srcs = torch.tensor([pair["src"] for pair in batch], dtype=torch.long, device=device)
    trgs = torch.tensor([pair["trg"] for pair in batch], dtype=torch.long, device=device)
    
    batch = {"src":srcs.T, "src_len":src_lens, "trg":trgs.T, "trg_len":trg_lens}
    return batch

 ## Non-attention

In [9]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout=0.5, bidirectional=True):
        super(Encoder, self).__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.gru = nn.GRU(emb_dim, hid_dim, n_layers, dropout=dropout, bidirectional=bidirectional)
        
    def forward(self, input_seqs, input_lengths, hidden):
        # input_seqs = [seq_len, batch]
        embedded = self.embedding(input_seqs)
        # embedded = [seq_len, batch, embed_dim]
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths, enforce_sorted=False)
        
        outputs, hidden = self.gru(packed, hidden)        
        outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        # outputs = [seq_len, batch, hid_dim * n directions]
        # output_lengths = [batch]
        return outputs, hidden
    
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout=0.5, bidirectional=True):
        super(Decoder, self).__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.gru = nn.GRU(emb_dim, hid_dim, n_layers, dropout=dropout, bidirectional=bidirectional)
        
        if bidirectional:
            self.fc_out = nn.Linear(hid_dim*2, output_dim)
        else:
            self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, token_inputs, hidden):
        # token_inputs = [batch]
        batch_size = token_inputs.size(0)
        embedded = self.dropout(self.embedding(token_inputs).view(1, batch_size, -1))
        # embedded = [1, batch, emb_dim]

        output, hidden = self.gru(embedded, hidden)
        # output = [1, batch,  n_directions * hid_dim]
        # hidden = [n_layers * n_directions, batch, hid_dim]
        
        output = self.fc_out(output.squeeze(0))
        output = self.softmax(output)
        # output = [batch, output_dim]
        return output, hidden
    
class Seq2Seq(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder, 
                 device, 
                 predict=False, 
                 basic_dict=None,
                 max_len=100
                 ):
        super(Seq2Seq, self).__init__()
        
        self.device = device

        self.encoder = encoder
        self.decoder = decoder

        self.predict = predict  # predict
        self.basic_dict = basic_dict
        self.max_len = max_len  # max length of output when translating

        self.enc_n_layers = self.encoder.gru.num_layers
        self.enc_n_directions = 2 if self.encoder.gru.bidirectional else 1
        self.dec_n_directions = 2 if self.decoder.gru.bidirectional else 1

        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        assert self.enc_n_directions >= self.dec_n_directions, \
            "If decoder is bidirectional, encoder must be bidirectional either!"
        
    def forward(self, input_batches, input_lengths, target_batches=None, target_lengths=None, teacher_forcing_ratio=0.5):
        # input_batches = target_batches = [seq_len, batch]
        batch_size = input_batches.size(1)
        
        BOS_token = self.basic_dict["<bos>"]
        EOS_token = self.basic_dict["<eos>"]
        PAD_token = self.basic_dict["<pad>"]

        # Initial
        encoder_hidden = torch.zeros(self.enc_n_layers*self.enc_n_directions, batch_size, self.encoder.hid_dim, device=self.device)
        
        # encoder_output = [seq_len, batch, hid_dim * n directions]
        # encoder_hidden = [n_layers*n_directions, batch, hid_dim]
        encoder_output, encoder_hidden = self.encoder(
            input_batches, input_lengths, encoder_hidden)

        decoder_input = torch.tensor([BOS_token] * batch_size, dtype=torch.long, device=self.device)
        if self.enc_n_directions == self.dec_n_directions:
            decoder_hidden = encoder_hidden
        else:
            L = encoder_hidden.size(0)
            decoder_hidden = encoder_hidden[range(0, L, 2)] + encoder_hidden[range(1, L, 2)]

        if self.predict:
            # input a sentence each time
            assert batch_size == 1, "batch_size of predict phase must be 1!"
            output_tokens = []

            while True:
                decoder_output, decoder_hidden = self.decoder(
                    decoder_input, decoder_hidden
                )
                # [1, 1]
                topv, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(1)
                output_token = topi.squeeze().detach().item()
                if output_token == EOS_token or len(output_tokens) == self.max_len:
                    break
                output_tokens.append(output_token)
            return output_tokens

        else:
            # training
            max_target_length = max(target_lengths)
            all_decoder_outputs = torch.zeros((max_target_length, batch_size, self.decoder.output_dim), device=self.device)

            for t in range(max_target_length):
                use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
                if use_teacher_forcing:
                    # decoder_output = [batch, output_dim]
                    # decoder_hidden = [n_layers*n_directions, batch, hid_dim]
                    decoder_output, decoder_hidden = self.decoder(
                        decoder_input, decoder_hidden
                    )
                    all_decoder_outputs[t] = decoder_output
                    decoder_input = target_batches[t]  
                else:
                    decoder_output, decoder_hidden = self.decoder(
                        decoder_input, decoder_hidden
                    )
                    # [batch, 1]
                    topv, topi = decoder_output.topk(1)
                    all_decoder_outputs[t] = decoder_output
                    decoder_input = topi.squeeze(1)
            
            loss_fn = nn.NLLLoss(ignore_index=PAD_token)
            loss = loss_fn(
                all_decoder_outputs.reshape(-1,self.decoder.output_dim ),  # [batch*seq_len, output_dim]
                target_batches.reshape(-1)                                                 # [batch*seq_len]
            )
            return loss

In [10]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def train(
    model,
    data_loader, 
    optimizer, 
    clip=1, 
    teacher_forcing_ratio=0.5, 
    print_every=None 
    ):
    model.predict = False
    model.train()

    if print_every == 0:
        print_every = 1

    print_loss_total = 0 
    start = time.time()
    epoch_loss = 0
    for i, batch in enumerate(data_loader):

        # shape = [seq_len, batch]
        input_batchs = batch["src"]
        target_batchs = batch["trg"]
        # list
        input_lens = batch["src_len"]
        target_lens = batch["trg_len"]
        
        optimizer.zero_grad()
        
        loss = model(input_batchs, input_lens, target_batchs, target_lens, teacher_forcing_ratio)
        print_loss_total += loss.item()
        epoch_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        if print_every and (i+1) % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('\tCurrent Loss: %.4f' % print_loss_avg)

    return epoch_loss / len(data_loader)

def evaluate(
    model,
    data_loader, 
    print_every=None
    ):
    model.predict = False
    model.eval()
    if print_every == 0:
        print_every = 1

    print_loss_total = 0  
    start = time.time()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            print(i)

            # shape = [seq_len, batch]
            input_batchs = batch["src"]
            target_batchs = batch["trg"]
            # list
            input_lens = batch["src_len"]
            target_lens = batch["trg_len"]

            loss = model(input_batchs, input_lens, target_batchs, target_lens, teacher_forcing_ratio=0)
            print_loss_total += loss.item()
            epoch_loss += loss.item()

            if print_every and (i+1) % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('\tCurrent Loss: %.4f' % print_loss_avg)

    return epoch_loss / len(data_loader)

def translate(
    model,
    sample, 
    idx2token=None
    ):
    model.predict = True
    model.eval()

    # shape = [seq_len, 1]
    input_batch = sample["src"]
    # list
    input_len = sample["src_len"]

    output_tokens = model(input_batch, input_len)
    output_tokens = [idx2token[t] for t in output_tokens]

    return "".join(output_tokens)

开始训练

In [12]:
INPUT_DIM = len(before2id)
OUTPUT_DIM = len(after2id)

# Hyperparameter
BATCH_SIZE = 32
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2 
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
LEARNING_RATE = 1e-4
N_EPOCHS = 2 #200
CLIP = 1

bidirectional = True
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT, bidirectional)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT, bidirectional)
model = Seq2Seq(enc, dec, device, basic_dict=basic_dict).to(device)
model.load_state_dict(torch.load("nonatte-model-dict-19.pt",map_location='cpu'))

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [14]:
# DataSet
train_set = TranslationDataset(before_num_data, after_num_data)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, collate_fn=padding_batch)
train_loader

In [15]:
len(train_loader)

4688

In [ ]:
best_valid_loss = float('inf')
N_EPOCHS = 10  #200

for epoch in range(N_EPOCHS):
    print('EPOCH'+str(epoch+20))
    
    start_time = time.time()
    train_loss = train(model, train_loader, optimizer, CLIP)
    valid_loss = evaluate(model, train_loader)
    end_time = time.time()

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'nonatte-model-dict-'+str(epoch+20)+'.pt')
        torch.save(model, 'nonatte-model-'+str(epoch+20)+'.pt')

    if epoch %1 == 0:
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        print(f'Epoch: {epoch+20:02} | Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Val. Loss: {valid_loss:.3f}')

In [17]:
random.seed(seed)
for i in random.sample(range(len(before_num_data)), 10):  
    before_tokens = list(filter(lambda x: x!=0, before_num_data[i])) 
    after_tokens = list(filter(lambda x: x!=3 and x!=0, after_num_data[i]))
    sentence = [id2before[t] for t in before_tokens]
    print("【原文】")
    print("".join(sentence))
    translation = [id2after[t] for t in after_tokens]
    print("【原文】")
    print("".join(translation))
    test_sample = {}
    test_sample["src"] = torch.tensor(before_tokens, dtype=torch.long, device=device).reshape(-1, 1)
    test_sample["src_len"] = [len(before_tokens)]
    print("【机器翻译】")
    print(translate(model, test_sample, id2after), end="\n\n")

【原文】
王涛更是集男团、男双、混双三顶桂冠于一身<eos>
【原文】
王涛更是集女团、女双、混双三顶桂冠于一身
【机器翻译】
王涛更是集女团、女双、双混三顶桂冠于一身

【原文】
虽然至今只留下三行字<eos>
【原文】
虽然至今只留下三行字
【机器翻译】
虽然至今只留下三行字

【原文】
又制造了新的政企不分、新的“父爱主义”<eos>
【原文】
又制造了新的政企不分、新的“母爱主义”
【机器翻译】
又制造了新的政企不分、新的“母爱主义”

【原文】
张庚先生有过深刻的论述：“我觉得梅兰芳的艺术风格<eos>
【原文】
张庚女士有过深刻的论述：“我觉得梅兰芳的艺术风格
【机器翻译】
张庚女士有过深刻的论述：“我觉得梅兰芳的艺术风格

【原文】
软科学日益受到广泛的重视<eos>
【原文】
软科学日益受到广泛的重视
【机器翻译】
软科学日益受到广泛的重视

【原文】
表示“均存在”或“均可找到”；R=(-∞<eos>
【原文】
表示“均存在”或“均可找到”；R=(-∞
【机器翻译】
表示“均存在”或“均可找到”；R=(-∞

【原文】
一望而知他们是特殊人物<eos>
【原文】
一望而知她们是特殊人物
【机器翻译】
一望而知她们是特殊人物

【原文】
他们要求的是“古人真诗”：真诗者<eos>
【原文】
她们要求的是“古人真诗”：真诗者
【机器翻译】
她们要求的是“古人真诗”：真诗者

【原文】
奥特吉先生、杨主席和吴学谦副总理在午餐会上先后祝酒<eos>
【原文】
奥特吉女士、杨主席和吴学谦副总理在午餐会上先后祝酒
【机器翻译】
奥特吉女士、杨主席和吴学副副总理在午午会上先后祝酒

【原文】
尤其1980年代期间和1990年代初在非洲和南美洲发现黄金和宝石时<eos>
【原文】
尤其1980年代期间和1990年代初在非洲和南美洲发现黄金和宝石时
【机器翻译】
尤其1980年代期间和1990年代初在非洲和南南美发现美黄黄金石石时



Attention

In [22]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout=0.5, bidirectional=True):
        super(Encoder, self).__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.gru = nn.GRU(emb_dim, hid_dim, n_layers, dropout=dropout, bidirectional=bidirectional)
        
    def forward(self, input_seqs, input_lengths, hidden):
        # input_seqs = [seq_len, batch]
        embedded = self.embedding(input_seqs)
        # embedded = [seq_len, batch, embed_dim]
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths, enforce_sorted=False)
        
        outputs, hidden = self.gru(packed, hidden)        
        outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        # outputs = [seq_len, batch, hid_dim * n directions]
        # output_lengths = [batch]
        return outputs, hidden
    
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)  # [seq_len, batch]

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)  # [seq_len, batch, hid_dim]
        return torch.sum(hidden * energy, dim=2)  # [seq_len, batch]

    def concat_score(self, hidden, encoder_output):
        # hidden.expand(encoder_output.size(0), -1, -1) -> [seq_len, batch, N]
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        # energy = [sql_len, batch, hidden_size]
        return torch.sum(self.v * energy, dim=2)  # [seq_len, batch]

    def forward(self, hidden, encoder_outputs):
        # hidden = [1, batch,  n_directions * hid_dim]
        # encoder_outputs = [seq_len, batch, hid dim * n directions]
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        attn_energies = attn_energies.t()  # [batch, seq_len]
 
        return F.softmax(attn_energies, dim=1).unsqueeze(1)  # softmax归一化# [batch, 1, seq_len]

class AttnDecoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers=1, dropout=0.5, bidirectional=True, attn_method="general"):
        super(AttnDecoder, self).__init__()

        self.output_dim = output_dim
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.dropout = dropout

        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(emb_dim, hid_dim, n_layers, dropout=dropout, bidirectional=bidirectional)
        
        if bidirectional:
            self.concat = nn.Linear(hid_dim * 2 * 2, hid_dim*2)
            self.out = nn.Linear(hid_dim*2, output_dim)
            self.attn = Attn(attn_method, hid_dim*2)
        else:
            self.concat = nn.Linear(hid_dim * 2, hid_dim)
            self.out = nn.Linear(hid_dim, output_dim)
            self.attn = Attn(attn_method, hid_dim)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, token_inputs, last_hidden, encoder_outputs):
        batch_size = token_inputs.size(0)
        embedded = self.embedding(token_inputs)
        embedded = self.embedding_dropout(embedded)
        embedded = embedded.view(1, batch_size, -1) # [1, B, hid_dim]

        gru_output, hidden = self.gru(embedded, last_hidden)
        # gru_output = [1, batch,  n_directions * hid_dim]
        # hidden = [n_layers * n_directions, batch, hid_dim]

        # encoder_outputs = [sql_len, batch, hid dim * n directions]
        attn_weights = self.attn(gru_output, encoder_outputs)
        # attn_weights = [batch, 1, sql_len]
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # [batch, 1, hid_dim * n directions]

        # LuongAttention
        gru_output = gru_output.squeeze(0) # [batch, n_directions * hid_dim]
        context = context.squeeze(1)       # [batch, n_directions * hid_dim]
        concat_input = torch.cat((gru_output, context), 1)  # [batch, n_directions * hid_dim * 2]
        concat_output = torch.tanh(self.concat(concat_input))  # [batch, n_directions*hid_dim]

        output = self.out(concat_output)  # [batch, output_dim]
        output = self.softmax(output)

        return output, hidden, attn_weights
    
class Seq2Seq(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder, 
                 device, 
                 predict=False, 
                 basic_dict=None,
                 max_len=100
                 ):
        super(Seq2Seq, self).__init__()
        
        self.device = device

        self.encoder = encoder
        self.decoder = decoder

        self.predict = predict
        self.basic_dict = basic_dict  
        self.max_len = max_len  

        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        assert encoder.gru.bidirectional == decoder.gru.bidirectional, \
            "Decoder and encoder must had same value of bidirectional attribute!"
        
    def forward(self, input_batches, input_lengths, target_batches=None, target_lengths=None, teacher_forcing_ratio=0.5):
        # input_batches = [seq_len, batch]
        # target_batches = [seq_len, batch]
        batch_size = input_batches.size(1)
        
        BOS_token = self.basic_dict["<bos>"]
        EOS_token = self.basic_dict["<eos>"]
        PAD_token = self.basic_dict["<pad>"]

        enc_n_layers = self.encoder.gru.num_layers
        enc_n_directions = 2 if self.encoder.gru.bidirectional else 1
        encoder_hidden = torch.zeros(enc_n_layers*enc_n_directions, batch_size, self.encoder.hid_dim, device=self.device)
        
        # encoder_outputs = [input_lengths, batch, hid_dim * n directions]
        # encoder_hidden = [n_layers*n_directions, batch, hid_dim]
        encoder_outputs, encoder_hidden = self.encoder(
            input_batches, input_lengths, encoder_hidden)

        decoder_input = torch.tensor([BOS_token] * batch_size, dtype=torch.long, device=self.device)
        decoder_hidden = encoder_hidden

        if self.predict:
            assert batch_size == 1, "batch_size of predict phase must be 1!"
            output_tokens = []

            while True:
                decoder_output, decoder_hidden, decoder_attn = self.decoder(
                    decoder_input, decoder_hidden, encoder_outputs
                )
                # [1, 1]
                topv, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(1).detach()
                output_token = topi.squeeze().detach().item()
                if output_token == EOS_token or len(output_tokens) == self.max_len:
                    break
                output_tokens.append(output_token)
            return output_tokens

        else:
            max_target_length = max(target_lengths)
            all_decoder_outputs = torch.zeros((max_target_length, batch_size, self.decoder.output_dim), device=self.device)

            for t in range(max_target_length):
                use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
                if use_teacher_forcing:
                    # decoder_output = [batch, output_dim]
                    # decoder_hidden = [n_layers*n_directions, batch, hid_dim]
                    decoder_output, decoder_hidden, decoder_attn = self.decoder(
                        decoder_input, decoder_hidden, encoder_outputs
                    )
                    all_decoder_outputs[t] = decoder_output
                    decoder_input = target_batches[t] 
                else:
                    decoder_output, decoder_hidden, decoder_attn = self.decoder(
                        decoder_input, decoder_hidden, encoder_outputs
                    )
                    # [batch, 1]
                    topv, topi = decoder_output.topk(1)
                    all_decoder_outputs[t] = decoder_output
                    decoder_input = topi.squeeze(1).detach() 
            
            loss_fn = nn.NLLLoss(ignore_index=PAD_token)
            loss = loss_fn(
                all_decoder_outputs.reshape(-1, self.decoder.output_dim),  # [batch*seq_len, output_dim]
                target_batches.reshape(-1)               # [batch*seq_len]
            )
            return loss

In [23]:
INPUT_DIM = len(before2id)
OUTPUT_DIM = len(after2id)
# Hyperparameter
BATCH_SIZE = 32
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 64 #512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
LEARNING_RATE = 1e-4
N_EPOCHS = 2 #200
CLIP = 1

bidirectional = True
attn_method = "general"
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT, bidirectional)
dec = AttnDecoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT, bidirectional, attn_method)

model = Seq2Seq(enc, dec, device, basic_dict=basic_dict).to(device)

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [24]:
len(before_num_data)

127135

In [25]:
# DataSet
train_set = TranslationDataset(before_num_data[0:500], after_num_data[0:500])
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, collate_fn=padding_batch)

In [ ]:
best_valid_loss = float('inf')
N_EPOCHS=1

for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss = train(model, train_loader, optimizer, CLIP)
    valid_loss = evaluate(model, train_loader)
    end_time = time.time()
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "attn-model1.pt")

    if epoch %10 == 0:
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Val. Loss: {valid_loss:.3f}')

In [27]:
model = Seq2Seq(enc, dec, device, basic_dict=basic_dict).to(device)
model.load_state_dict(torch.load("good-Areverse-model-0.pt",map_location='cpu'))

<All keys matched successfully>

In [ ]:
random.seed(seed)
for i in random.sample(range(len(before_num_data)), 10):
    before_tokens = list(filter(lambda x: x!=0, before_num_data[i])) 
    after_tokens = list(filter(lambda x: x!=3 and x!=0, after_num_data[i]))  
    sentence = [id2before[t] for t in before_tokens]
    print("【原文】")
    print("".join(sentence))
    translation = [id2after[t] for t in after_tokens]
    print("【原文】")
    print("".join(translation))
    test_sample = {}
    test_sample["src"] = torch.tensor(before_tokens, dtype=torch.long, device=device).reshape(-1, 1)
    test_sample["src_len"] = [len(before_tokens)]
    print("【机器翻译】")
    print(translate(model, test_sample, id2after), end="\n\n")